In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('https://cmip6.storage.googleapis.com/cmip6-zarr-consolidated-stores.csv')

In [ ]:
# define a simple search on keywords
def search_df(df, verbose= False, **search):
    "search by keywords - if list, then match exactly, otherwise match as substring"
    keys = ['activity_id','institution_id','source_id','experiment_id','member_id', 'table_id', 'variable_id', 'grid_label']
    d = df
    for skey in search.keys():
        
        if isinstance(search[skey], str):  # match a string as a substring
            d = d[d[skey].str.contains(search[skey])]
        else:
            dk = []
            for key in search[skey]:       # match a list of strings exactly
                dk += [d[d[skey]==key]]
            d = pd.concat(dk)
            keys.remove(skey)
    if verbose:
        for key in keys:
            print(key,' = ',list(d[key].unique()))      
    return d

In [ ]:
dfp = search_df(df, experiment_id=['historical','1pctCO2'], table_id='Omon', variable_id=['tos','o2','chl','dissic'], grid_label=['gn'])

In [ ]:
dm = dfp[['experiment_id','source_id','variable_id','member_id',]].groupby(['experiment_id','source_id','variable_id']).nunique()[['member_id']]

table = pd.DataFrame.pivot_table(dm, values='member_id', index=['experiment_id','source_id'],
                                 columns=['variable_id'], aggfunc=np.sum, fill_value=0)
table

In [ ]:
table[(table.chl>0)&(table.dissic>0)&(table.tos>0)]